In [2]:
import requests
import os
from google.auth import default
from google.auth.transport.requests import Request
from google.cloud import optimization_v1
import googlemaps
from dotenv import load_dotenv

load_dotenv()
proxy_address = "http://127.0.0.1:7890"
# Set the environment variables to specify the proxy
os.environ["HTTP_PROXY"] = proxy_address
os.environ["HTTPS_PROXY"] = proxy_address
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/kaicheng/ProjectsFormal/wheelTrans/secrets/key.json"
os.environ["PROJECT_ID"] = "elderlyhometransportation"
googleMapKey = os.getenv("GOOGLE_MAP_API_KEY")

ROOT_PATH = os.path.dirname(os.path.abspath(''))
ROOT_PATH


'/Users/kaicheng/ProjectsFormal/wheelTrans/src'

In [3]:
def call_sync_api() -> None:
    request_file_name = "resources/request.json"
    fleet_routing_client = optimization_v1.FleetRoutingClient()

    with open(request_file_name) as f:
        fleet_routing_request = optimization_v1.OptimizeToursRequest.from_json(f.read())
        fleet_routing_request.parent = f"projects/elderlyhometransportation"
        fleet_routing_response = fleet_routing_client.optimize_tours(
            fleet_routing_request, timeout=10
        )
        return fleet_routing_response


def get_direction(origin, destination, waypoints):
    waypoints_str = "|".join(["optimize:true"] + waypoints)
    base_url = "https://maps.googleapis.com/maps/api/directions/json"
    google_map_api_key = os.getenv("GOOGLE_MAP_API_KEY")

    params = {
        "destination": destination,
        "origin": origin,
        "waypoints": waypoints_str,
        "key": google_map_api_key
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None


def get_geocoding(gmaps, address):
    result = gmaps.geocode(address)
    if not result or len(result) != 1:
        print(f"Failed to get geocoding for {address}")
        return None
    location = result[0]['geometry']['location']
    lat = location["lat"]
    lng = location["lng"]
    return [lat, lng]


origin = "8408 Garvey Ave. #101 Rosemead, CA 91770"
destination = "8408 Garvey Ave. #101 Rosemead, CA 91770"
address1 = "3843 Maxson Road #226 El Monte, CA 91732"
address2 = "119 Garcelon Ave Apt B Monterey Park, CA 91754"
waypoints = [origin, address1, address2, destination]
# get_direction(origin, destination, waypoints, destination)
# get_geocoding(origin)
# get_geocoding(address1)
# get_geocoding(address2)
# gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
# coordinates = []
# for location in waypoints:
#     coordinates.append(get_geocoding(gmaps, location))
# print(coordinates)
# response = call_sync_api()


In [9]:
import pandas

df = pandas.read_excel("resources/TransportList.xlsx")
addresses = addresses = df['Address'].dropna().tolist()
#dev short
addresses = addresses[:10]
addresses

['3843 Maxson Road #226  El Monte, CA 91732',
 '119 Garcelon Ave Apt B  Monterey Park, CA 91754',
 '119 Garcelon Ave Apt B  Monterey Park, CA 91754',
 '3843 Maxson Rd Apt 314  El Monte, CA 91732',
 '3843 Maxson Rd Apt 314  El Monte, CA 91732',
 '2417 Angelus Ave #306 6266880563 Rosemead, CA 91770',
 '201 1/2 W Newmark Ave  Monterey Park, CA 91755',
 '209 W. Camino Real Ave  Arcadia, CA 91007',
 '209 W. Camino Real Ave  Arcadia, CA 91007',
 '3436 Tyler Ave #409  El Monte, CA 91731']

In [73]:
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
coordinates = []
validAddresses = []
invalidAddresses = []
for address in addresses:
    latlng = get_geocoding(gmaps, address)
    if latlng:
        coordinates.append(latlng)
        validAddresses.append(address)
    else:
        print(f"Failed to get geocoding for {address}")
        invalidAddresses.append(address)

coordinates

[[34.072393, -118.0102374],
 [34.0651666, -118.12373],
 [34.0651666, -118.12373],
 [34.072393, -118.0102374],
 [34.072393, -118.0102374],
 [34.0564491, -118.0874686],
 [34.0600187, -118.1244518],
 [34.122069, -118.0391543],
 [34.122069, -118.0391543],
 [34.0709251, -118.0357052]]

In [76]:
validAddresses, invalidAddresses

(['3843 Maxson Road #226  El Monte, CA 91732',
  '119 Garcelon Ave Apt B  Monterey Park, CA 91754',
  '119 Garcelon Ave Apt B  Monterey Park, CA 91754',
  '3843 Maxson Rd Apt 314  El Monte, CA 91732',
  '3843 Maxson Rd Apt 314  El Monte, CA 91732',
  '2417 Angelus Ave #306 6266880563 Rosemead, CA 91770',
  '201 1/2 W Newmark Ave  Monterey Park, CA 91755',
  '209 W. Camino Real Ave  Arcadia, CA 91007',
  '209 W. Camino Real Ave  Arcadia, CA 91007',
  '3436 Tyler Ave #409  El Monte, CA 91731'],
 [])

In [11]:
from resources.vehicles import vehicles
import json  

shipments = list(map(lambda latlng:
                     {
                         "loadDemands": {
                             "weight": {
                                 "amount": "1"
                             }
                         },
                         "pickups": [
                             {
                                 "arrivalLocation": {
                                     "latitude": latlng[0],
                                     "longitude": latlng[1]
                                 },
                                 "duration": "150s",
                                #  "timeWindows": [
                                #      {
                                #          "endTime": "1970-01-01T00:33:20Z",
                                #          "startTime": "1970-01-01T00:16:40Z"
                                #      }
                                #  ]
                             }
                         ]
                     }
                     , coordinates))
requestDict = {
    "parent": "projects/elderlyhometransportation",
    "model": {
        "shipments": shipments,
        "vehicles": vehicles
    }
}
requestJson = json.dumps(requestDict)

requestJson

'{"parent": "projects/elderlyhometransportation", "model": {"shipments": [{"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.072393, "longitude": -118.0102374}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.0651666, "longitude": -118.12373}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.0651666, "longitude": -118.12373}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.072393, "longitude": -118.0102374}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.072393, "longitude": -118.0102374}, "duration": "150s"}]}, {"loadDemands": {"weight": {"amount": "1"}}, "pickups": [{"arrivalLocation": {"latitude": 34.0564491, "longitude": -118.0874686}, "duration": "150s"}]}, {"loadDemands":

In [14]:
fleet_routing_client = optimization_v1.FleetRoutingClient()
fleetOptimizationRequest = optimization_v1.OptimizeToursRequest.from_json(requestJson)

response = fleet_routing_client.optimize_tours(
    fleetOptimizationRequest, timeout=10
)
response

routes {
  vehicle_start_time {
  }
  vehicle_end_time {
    seconds: 4643
  }
  visits {
    shipment_index: 4
    is_pickup: true
    start_time {
      seconds: 794
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
    }
    arrival_loads {
      type_: "weight"
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 3
    is_pickup: true
    start_time {
      seconds: 944
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 150
    }
    arrival_loads {
      type_: "weight"
      value: 1
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 2
    is_pickup: true
    start_time {
      seconds: 1938
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 1367
    }
    arrival_loads {
      type_: "weight"
      val

In [77]:
optimizedRoutes = []
for vehicleRoute in response.routes:
    displayWaypoints=[]
    latlngWaypoints=[]
    for index, visit in enumerate(vehicleRoute.visits):
        shipmentId = visit.shipment_index
        displayWaypoints.append({
            'arrival': visit.start_time,
            'address': validAddresses[shipmentId]
        })
        latlngWaypoints.append(coordinates[shipmentId])
    optimizedRoutes.append({
        'vehicle': vehicleRoute.vehicle_index or 0,
        'order':latlngWaypoints,
        'displayWaypoints': displayWaypoints
        })
optimizedRoutes

[{'vehicle': 0,
  'order': [[34.072393, -118.0102374],
   [34.072393, -118.0102374],
   [34.0651666, -118.12373],
   [34.072393, -118.0102374],
   [34.0651666, -118.12373]],
  'displayWaypoints': [{'arrival': DatetimeWithNanoseconds(1970, 1, 1, 0, 13, 14, tzinfo=datetime.timezone.utc),
    'address': '3843 Maxson Rd Apt 314  El Monte, CA 91732'},
   {'arrival': DatetimeWithNanoseconds(1970, 1, 1, 0, 15, 44, tzinfo=datetime.timezone.utc),
    'address': '3843 Maxson Rd Apt 314  El Monte, CA 91732'},
   {'arrival': DatetimeWithNanoseconds(1970, 1, 1, 0, 32, 18, tzinfo=datetime.timezone.utc),
    'address': '119 Garcelon Ave Apt B  Monterey Park, CA 91754'},
   {'arrival': DatetimeWithNanoseconds(1970, 1, 1, 0, 50, tzinfo=datetime.timezone.utc),
    'address': '3843 Maxson Road #226  El Monte, CA 91732'},
   {'arrival': DatetimeWithNanoseconds(1970, 1, 1, 1, 6, 34, tzinfo=datetime.timezone.utc),
    'address': '119 Garcelon Ave Apt B  Monterey Park, CA 91754'}]},
 {'vehicle': 1,
  'order'

In [71]:
for route in optimizedRoutes:
    print(route)
    print(route['vehicle'])


{'vehicle': 0, 'order': {'latlng': [[34.072393, -118.0102374], [34.072393, -118.0102374], [34.0651666, -118.12373], [34.072393, -118.0102374], [34.0651666, -118.12373]], 'address': ['3843 Maxson Rd Apt 314  El Monte, CA 91732', '3843 Maxson Rd Apt 314  El Monte, CA 91732', '119 Garcelon Ave Apt B  Monterey Park, CA 91754', '3843 Maxson Road #226  El Monte, CA 91732', '119 Garcelon Ave Apt B  Monterey Park, CA 91754']}}
0
{'vehicle': 1, 'order': {'latlng': [[34.0709251, -118.0357052], [34.122069, -118.0391543], [34.122069, -118.0391543], [34.0564491, -118.0874686], [34.0600187, -118.1244518]], 'address': ['3436 Tyler Ave #409  El Monte, CA 91731', '209 W. Camino Real Ave  Arcadia, CA 91007', '209 W. Camino Real Ave  Arcadia, CA 91007', '2417 Angelus Ave #306 6266880563 Rosemead, CA 91770', '201 1/2 W Newmark Ave  Monterey Park, CA 91755']}}
1


In [79]:
import polyline

def generate_static_map(origin, destination, waypoints, filename):
    # Create directions request with waypoints
    directions = gmaps.directions(origin, destination, waypoints=waypoints, mode="driving")

    # Extract route polyline from directions response
    route = directions[0]['overview_polyline']['points']
    decoded_route = polyline.decode(route)
    waypoints_polyline = '|'.join([f"{lat},{lng}" for lat, lng in waypoints])

    # Static Maps API request to generate map image with route and waypoints
    static_map_url = f'https://maps.googleapis.com/maps/api/staticmap?size=600x400&path=enc:{route}&markers=color:red|{waypoints_polyline}&key={googleMapKey}'
    print (static_map_url)
    # Download the map image
    map_image = requests.get(static_map_url)

    # Save the map image to a file
    with open('output/'+filename, 'wb') as f:
        f.write(map_image.content)

origin = (34.0623483, -118.0859541)  # Los Angeles, CA
destination = (34.0623483, -118.0859541)  # Los Angeles, CA
for route in optimizedRoutes:
    waypoints = route['order']
    print(waypoints)
    vehicleInd = route['vehicle']
    generate_static_map(origin=origin, destination=destination, waypoints=waypoints, filename=f'map_with_route{vehicleInd}.png' )



[[34.072393, -118.0102374], [34.072393, -118.0102374], [34.0651666, -118.12373], [34.072393, -118.0102374], [34.0651666, -118.12373]]
https://maps.googleapis.com/maps/api/staticmap?size=600x400&path=enc:ux{nEjvvoU}@?@hAKlBArBLrA@lCIvA?rB?pBe@?yA@yE@wUFmJEs@KEEiA{@gAMyBAm@u@IgA?AWO?aBCyDAiDEgNGoTQkp@YyqAK_\V}NTuBfAiF`DwJpFmPbBgG~@eGrAkRhA_Sp@mXE_XRiBDgBRk@b@Ud@D^^Lh@QdAaApAM?m@UUUg@e@sCuCsLyLqCoCuDiEaDyDw@c@s@_@Yq@UcEG{@]kEK}CSgDq@{JcBiY}A_WMyAAmBnHtCnIfDn@TXLo@fCn@gCYMo@UoCgA_E_BoHuCM]OqCg@}Hc@wGy@iJsBeNSeBEu@d@A`AAvB?~BHbCVbBXh@GtD~@xBd@h@`@rOvDdDt@jK~@tFh@dBpAv@|C^jDXXHdAHtA^xEp@xINbEOtFm@xFgCjSaChNgAzDgA|Ci@bC{@fCsAbEmBdL[zEEfGXzYDtXm@jWwCze@iBxKkChIaBvEs@pAq@jBiDdJoAbFk@~EKrCXvVH~IPn_@Rfp@XzcAVr~@H~VQf]N`U\lSJh[HtWBvH@vHUl@GjC?tBGfDORaAHkDAQQEm@^]xEApA?`A?dICvBI~DCnD?`JEr@AfAC?bD?cDgABs@@aJDoD?_EBaAKaBA[k@Yc@kDGk@a@e@cBAaHMk^Q{k@g@olBcAayDI{XHqS`@{EfAiFpA{D`DwJzEeOhAsF|@gJpB}Zx@mYB}REmGRiBNgC\[`@Eb@NXj@@f@Op@aApAM?m@UUUg@e@e@g@{G{GoDsDsHsHyDeFiCuBs@_@Yq@UcEe@gGMmEs@cJ}AcW}@}Ou@wLMy

In [10]:
routes = response.routes
routesPb0 = response.routes[0]
routesPb1 = response.routes[1]
a = list(map(lambda x: {"vehicleIndex": x.vehicle_index}, routes))
